In [182]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
from mlxtend.frequent_patterns import apriori, association_rules

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [184]:
# load the data set ans show the first five transaction
url = "https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df = pd.read_csv(url)
df.head(5)

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


In [185]:
#michan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       315 non-null    object
 1   1       285 non-null    object
 2   2       245 non-null    object
 3   3       187 non-null    object
 4   4       133 non-null    object
 5   5       71 non-null     object
 6   6       41 non-null     object
dtypes: object(7)
memory usage: 17.4+ KB


# Get the set of product that has been purchased


In [186]:
products = set()
for i in df:
  products.update(df[i].unique())
print(products)

{nan, 'Diaper', 'Pencil', 'Bagel', 'Eggs', 'Milk', 'Bread', 'Meat', 'Wine', 'Cheese'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [187]:
#create an itemset based on the products
itemset = set(products)
# encoding the feature
encoded = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded.append(labels)
encoded[0]

{nan: 0,
 'Bagel': 0,
 'Milk': 0,
 'Diaper': 1,
 'Pencil': 1,
 'Eggs': 1,
 'Bread': 1,
 'Meat': 1,
 'Wine': 1,
 'Cheese': 1}

In [188]:
  # create new dataframe from the encoded features
df_2 = pd.DataFrame(encoded)
  # show the new dataframe
df_2.head(10)

,NaN,Bagel,Milk,Diaper,Pencil,Eggs,Bread,Meat,Wine,Cheese
0,0,0,0,1,1,1,1,1,1,1
1,0,0,1,1,1,0,1,1,1,1
2,1,0,1,0,0,1,0,1,1,1
3,1,0,1,0,0,1,0,1,1,1
4,1,0,0,0,1,0,0,1,1,0
5,0,1,1,1,1,1,1,0,1,0
6,1,0,0,0,1,1,0,0,1,1
7,1,1,1,1,1,0,1,0,0,0
8,1,0,1,1,0,1,1,0,1,1
9,0,1,0,1,1,1,0,1,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [189]:
df_2.columns = df_2.columns.fillna('to_drop')
df_2.drop('to_drop', axis = 1, inplace = True)

In [190]:
df_2.head(5)

,Bagel,Milk,Diaper,Pencil,Eggs,Bread,Meat,Wine,Cheese
0,0,0,1,1,1,1,1,1,1
1,0,1,1,1,0,1,1,1,1
2,0,1,0,0,1,0,1,1,1
3,0,1,0,0,1,0,1,1,1
4,0,0,0,1,0,0,1,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [196]:
freq_items = apriori(df_2, min_support=0.2, use_colnames=True)
freq_items.head(33)

,support,itemsets
0,0.425397,(Bagel)
1,0.501587,(Milk)
2,0.406349,(Diaper)
3,0.361905,(Pencil)
4,0.438095,(Eggs)
5,0.504762,(Bread)
6,0.476190,(Meat)
7,0.438095,(Wine)
8,0.501587,(Cheese)
9,0.225397,"(Bagel, Milk)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [199]:
ass_rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
ass_rules.head(14)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
2,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
3,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
4,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
5,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
6,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
7,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
8,"(Cheese, Milk)",(Meat),0.304762,0.476190,0.203175,0.666667,1.400000,0.058050,1.571429
9,"(Cheese, Meat)",(Milk),0.323810,0.501587,0.203175,0.627451,1.250931,0.040756,1.337845
